<a href="https://colab.research.google.com/github/Stephanie9606/Python_Multimodal_Deep_Learning/blob/main/FineTune_VGG_Memes_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [4]:
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

In [5]:
# Set up the device for GPU usage
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
# connect to google drive for files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Pre-processing

In [ ]:
# unzip image file
!unzip /content/drive/MyDrive/memes_image.zip

In [10]:
memesDF = pd.read_csv('/content/drive/MyDrive/top5_memes_tidy.tsv', sep='\t')

In [11]:
memesDF = memesDF.iloc[:, 2:9]

In [13]:
memesDF['AltText'] = memesDF['AltText'].astype(pd.StringDtype())
memesDF['MemeLabel'] = memesDF['MemeLabel'].astype(pd.StringDtype())
memesDF['CaptionText'] = memesDF['CaptionText'].astype(pd.StringDtype())
memesDF['HashId'] = memesDF['HashId'].astype(pd.StringDtype())
memesDF['ImageURL'] = memesDF['ImageURL'].astype(pd.StringDtype())
memesDF['ImagePath'] = memesDF['ImagePath'].astype(pd.StringDtype())
memesDF['Image'] = memesDF['Image'].astype(pd.StringDtype())

In [14]:
# Only get two label for Binary Classification
meme2cDF = memesDF[(memesDF['MemeLabel'] == "Scared Cat") | (memesDF['MemeLabel'] == "Who Killed Hannibal")]

In [ ]:
meme2cDF.head(3)

In [16]:
# train test split
from sklearn.model_selection import train_test_split

me_trainDF, me_testDF = train_test_split(meme2cDF, test_size=0.2, random_state=15)
me_trainDF.index

Int64Index([3025,  363,  222, 3160,  100,   51, 2855, 2651,  135, 2370,
            ...
            1047,  927,  749, 3115,  221,  943,  630, 3183, 2789, 3254],
           dtype='int64', length=1861)

# Load Pre-trained Model

In [22]:
#tf.random.set_seed(45)

# Load original model with pretrained weights from imagenet
base_model = applications.VGG16(weights="imagenet")

# Freeze base model
base_model.trainable = False

553476096/553467096 [==============================] - 5s 0us/step


In [ ]:
#base_model.summary()

In [ ]:
# Create new model on top
img_height = 224
img_width = 224
inputs = Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
outputs = Dense(1, activation="softmax")(x) # we want to output probabilities for both classes
model = Model(inputs, outputs)
model.summary()

In [ ]:
# Try deeper model on top
img_height = 224
img_width = 224
inputs = Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)

hidden1 = Flatten(name="flatten")(x)
hidden2 = Dense(512, activation="relu")(hidden1)
hidden3 = Dropout(0.5)(hidden2)

outputs = Dense(1, activation="softmax")(hidden3) # output probabilities for both classes
model = Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 1000)              138357544 
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 512)               512512    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 138,870,569
Trainable params: 513,025
Non-train

In [ ]:
base_model.layers.pop()
model2 = Model(base_model.input, base_model.layers[-2].output)
#model2.summary()
model2 = model2(inputs, training=False)

In [ ]:
# pop base_model to 4096 output feature layer
img_height = 224
img_width = 224
inputs = Input(shape=(img_height, img_width, 3))

base_model.layers.pop()
model2 = Model(base_model.input, base_model.layers[-2].output)
#model2.summary()
x = model2(inputs, training=False)

hidden1 = Flatten(name="flatten")(x)
hidden2 = Dense(512, activation="relu")(hidden1)
hidden3 = Dropout(0.5)(hidden2)

outputs = Dense(1, activation="softmax")(hidden3) # output probabilities for both classes
model = Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 model_1 (Functional)        (None, 4096)              134260544 
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               2097664   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 513       
                                                                 
Total params: 136,358,721
Trainable params: 2,098,177
Non-t

In [17]:
 # Loading training data
img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(preprocessing_function=applications.vgg16.preprocess_input, rescale=1./255.,validation_split=0.25)
train_generator = train_datagen.flow_from_dataframe(dataframe=me_trainDF, 
                                                    directory=None, 
                                                    x_col="ImagePath", 
                                                    y_col="MemeLabel", 
                                                    subset="training", 
                                                    seed = 55, 
                                                    class_mode="binary", 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size)

valid_generator = train_datagen.flow_from_dataframe(dataframe=me_trainDF, 
                                                    directory=None, 
                                                    x_col="ImagePath", 
                                                    y_col="MemeLabel", 
                                                    subset="validation", 
                                                    seed = 55, 
                                                    class_mode="binary", 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size)

print(f"Class labels: {train_generator.class_indices}")

#step_size = train_generator.n//train_generator.batch_size

Found 1396 validated image filenames belonging to 2 classes.
Found 465 validated image filenames belonging to 2 classes.
Class labels: {'Scared Cat': 0, 'Who Killed Hannibal': 1}


In [18]:
# Hyperparameter tuning

def create_model(params):
  img_height = 224
  img_width = 224
  inputs = Input(shape=(img_height, img_width, 3))

  base_model = applications.VGG16(weights="imagenet")
#  base_model.layers.pop()
  base_model = Model(base_model.input, base_model.layers[-2].output)
  x = base_model(inputs, training=False)
  hidden1 = Dense(params['units1'], activation = 'relu')(x)
  hidden2 = Dropout(params['dropout1'])(hidden1)

  outputs = Dense(1, activation = 'softmax')(hidden2)

  model = Model(inputs, outputs)
  return model

In [19]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
from keras.layers.core import Dense, Dropout, Activation
import sys

In [ ]:
x = train_generator
validation_data = valid_generator

# Define NN
def f_nn(params):   
    model = create_model(params)
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

    md_history = model.fit(x, validation_data = validation_data, epochs=10, batch_size=32, verbose = 2)
#    model.fit(x, epochs=params['num_epochs'], batch_size=params['batch_size'], verbose = 2)

    loss, acc = model.evaluate(validation_data, verbose = 0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model} 

# Search space
space = {'units1': hp.uniform('units1', 64,1024),
         'dropout1': hp.uniform('dropout1', .25,.75)
#         'batch_size' : hp.uniform('batch_size', 28,128)
        }

# Trials
trials = Trials()

best = fmin(f_nn, space, algo=tpe.suggest, max_evals=16, trials=trials)
print('best: ', best)

In [ ]:
!pip install mlflow

In [ ]:
import mlflow.tensorflow

In [ ]:
# Check out the MLflow UI as this runs
mlflow.tensorflow.autolog(every_n_iter=2)

In [ ]:
# optimizer and metric
model.compile(loss="binary_crossentropy", optimizer=Adam(lr=0.005), metrics=["accuracy"]) 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Train the model 
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

model.fit(train_generator,
          steps_per_epoch=step_size_train,
          validation_data=valid_generator,
          validation_steps=step_size_valid,
          epochs=10,
          verbose=2 # show: epoch 1/10
)

2022/03/27 20:08:04 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9d596fa92ef545fd832512bc0ae3ab89', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/10
43/43 - 40s - loss: 1.5578 - accuracy: 0.5183 - val_loss: 1.3773e-04 - val_accuracy: 0.5112 - 40s/epoch - 932ms/step
Epoch 2/10
43/43 - 21s - loss: 0.0346 - accuracy: 0.5191 - val_loss: 8.7123e-05 - val_accuracy: 0.5067 - 21s/epoch - 488ms/step
Epoch 3/10
43/43 - 22s - loss: 0.0337 - accuracy: 0.5205 - val_loss: 1.7694e-04 - val_accuracy: 0.5179 - 22s/epoch - 512ms/step
Epoch 4/10
43/43 - 26s - loss: 0.0073 - accuracy: 0.5198 - val_loss: 0.0029 - val_accuracy: 0.5223 - 26s/epoch - 605ms/step
Epoch 5/10
43/43 - 21s - loss: 0.0099 - accuracy: 0.5213 - val_loss: 3.4809e-04 - val_accuracy: 0.5112 - 21s/epoch - 490ms/step
Epoch 6/10
43/43 - 21s - loss: 0.0073 - accuracy: 0.5198 - val_loss: 0.0038 - val_accuracy: 0.5134 - 21s/epoch - 491ms/step
Epoch 7/10
43/43 - 26s - loss: 0.0146 - accuracy: 0.5198 - val_loss: 4.9030e-05 - val_accuracy: 0.5201 - 26s/epoch - 612ms/step
Epoch 8/10
43/43 - 21s - loss: 0.0078 - accuracy: 0.5198 - val_loss: 1.2050e-04 - val_accuracy: 0.5179 - 21s/epo

2022/03/27 20:13:39 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.2+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). MLflow logged a pip requirement for this package as 'jaxlib==0.3.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


In [ ]:
# Evaluate model on test set
test_datagen = ImageDataGenerator(preprocessing_function=applications.vgg16.preprocess_input, rescale=1./255.)

In [ ]:
# Evaluate
batch_size = 32

test_generator = test_datagen.flow_from_dataframe(
  dataframe=me_testDF, 
  directory=None, 
  x_col="ImagePath", 
  y_col="MemeLabel", 
  class_mode="binary", 
  target_size=(img_height, img_width),
  shuffle=False,
  batch_size=batch_size
)

step_size_test = test_generator.n//test_generator.batch_size

eval_results = model.evaluate(test_generator, steps=step_size_test)
print(f"Loss: {eval_results[0]}. Accuracy: {eval_results[1]}")

Found 466 validated image filenames belonging to 2 classes.
14/14 [==============================] - 6s 413ms/step - loss: 0.0029 - accuracy: 0.5112
Loss: 0.0029087255243211985. Accuracy: 0.5111607313156128


In [ ]:
# hyperpaprameter tuning 

466